In [43]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [47]:
# The csv was originally scraped from Wine Enthusiast magazine
# Pull in wine data from a csv.  
# Cleaned data by removing entries where "America = province" (should be state)

In [48]:
winemag = "./csv/winemag-data-130k.csv"
winemag_data = pd.read_csv(winemag)
winemag_data = winemag_data[winemag_data.province != 'America']

In [50]:
# Remove unnecessary/unused columns from winemag_data
# The columns listed were not required for analysis/plotting

In [51]:
winemag_df = winemag_data.drop(columns=["Unnamed: 0","description","region_1","region_2","taster_name","taster_twitter_handle","title"])
winemag_df.head()

,country,designation,points,price,province,variety,winery
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [ ]:
# Check row counts to ensure that we have the correct data

In [52]:
winemag_all_rows = winemag_df
winemag_all_rows.count()

country        129813
designation     92437
points         129876
price          120880
province       129813
variety        129875
winery         129876
dtype: int64

In [ ]:
# Have a look at the country counts and averages

In [34]:
by_country_count = winemag_all_rows.groupby("country").count()
by_country_count
by_country_mean = winemag_all_rows.groupby("country").mean().reset_index() #get country to show
by_country_mean.head()

,country,points,price
0,Argentina,86.710263,24.510117
1,Armenia,87.500000,14.500000
2,Australia,88.580507,35.437663
3,Austria,90.101345,30.762772
4,Bosnia and Herzegovina,86.500000,12.500000


In [ ]:
# Export to csv and load in excel to view for additional verification

In [53]:
winemag_all_rows.to_csv("./csv/winemag_cleaned.csv")

In [ ]:
# Connect to Postgresql relational database

In [54]:
connection_string = "postgres:postgres@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Create table for data import using sqlalchemy

In [37]:
engine.execute('CREATE TABLE "winedata" ('
'id INTEGER PRIMARY KEY,'
'country varchar,'
'designation varchar,'
'points int,'
'price float,'
'province varchar,'
'variety varchar,'
'winery varchar'
');')

ProgrammingError: (psycopg2.errors.DuplicateTable) relation "winedata" already exists

[SQL: CREATE TABLE "winedata" (id INTEGER PRIMARY KEY,country varchar,designation varchar,points int,price float,province varchar,variety varchar,winery varchar);]
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
# Verify that table was created

In [55]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# Specify the rowid in the dataframe & view the first five lines

In [39]:
winemag_all_rows.index.name = 'id'
winemag_all_rows.head()

,country,designation,points,price,province,variety,winery
id,,,,,,,
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Pinot Noir,Sweet Cheeks


In [ ]:
# Import the data from the pandas dataframe into the Postgresql table we created above

In [40]:
winemag_all_rows.to_sql(name='winedata', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "winedata_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO winedata (id, country, designation, points, price, province, variety, winery) VALUES (%(id)s, %(country)s, %(designation)s, %(points)s, %(price)s, %(province)s, %(variety)s, %(winery)s)]
[parameters: ({'id': 0, 'country': 'Italy', 'designation': 'Vulkà Bianco', 'points': 87, 'price': None, 'province': 'Sicily & Sardinia', 'variety': 'White Blend', 'winery': 'Nicosia'}, {'id': 1, 'country': 'Portugal', 'designation': 'Avidagos', 'points': 87, 'price': 15.0, 'province': 'Douro', 'variety': 'Portuguese Red', 'winery': 'Quinta dos Avidagos'}, {'id': 2, 'country': 'US', 'designation': None, 'points': 87, 'price': 14.0, 'province': 'Oregon', 'variety': 'Pinot Gris', 'winery': 'Rainstorm'}, {'id': 3, 'country': 'US', 'designation': 'Reserve Late Harvest', 'points': 87, 'price': 13.0, 'province': 'Michigan', 'variety': 'Riesli

In [ ]:
# View first five lines imported data directly from the Postgresql table

In [41]:
engine.execute('SELECT * FROM winedata LIMIT 5').fetchall()

[(0, 'Italy', 'Vulkà Bianco', 87, None, 'Sicily & Sardinia', 'White Blend', 'Nicosia'),
 (1, 'Portugal', 'Avidagos', 87, 15.0, 'Douro', 'Portuguese Red', 'Quinta dos Avidagos'),
 (2, 'US', None, 87, 14.0, 'Oregon', 'Pinot Gris', 'Rainstorm'),
 (3, 'US', 'Reserve Late Harvest', 87, 13.0, 'Michigan', 'Riesling', 'St. Julian'),
 (4, 'US', "Vintner's Reserve Wild Child Block", 87, 65.0, 'Oregon', 'Pinot Noir', 'Sweet Cheeks')]